# Notebook Outline
- Holding companies: How many holding companies are there? What is the most census blocks any company serves? What are the fewest any holding company serves?
- Broadband type(tech_code): What are the different types? What is the most popular? Which provides the fastest service?
- Speed: Median Up/Down (edited)

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
def query_bq(query):
    return pd.read_gbq(query, project_id='fccdsicapstone-218522', dialect='standard')

## Holding companies:

In [29]:
# distinct within subquery ensures to avoid situations where a hoco
# offers multiple broadband types to the same census block

query = """
SELECT hoco_final, COUNT(*) as ct FROM(
  SELECT
      DISTINCT(CONCAT(hoco_final, block_code)) as hoco_block,
      hoco_final
  FROM broadband.fcc
  WHERE file_date = 201412)
GROUP BY hoco_final
ORDER BY ct DESC
"""
df = query_bq(query)

There are 5 Holding Comapnies which serve over 1 million census block

In [30]:
df.iloc[0:10]

,hoco_final,ct
0,AT&T Inc.,2087234
1,Comcast Corporation,1606962
2,Time Warner Cable Inc.,1232019
3,"CenturyLink, Inc.",1126478
4,Verizon Communications Inc.,1017697
5,Charter Communications,841167
6,"JAB Wireless, Inc.",668451
7,"King Street PCS, Inc.",479896
8,Frontier Communications Corporation,438900
9,"Cox Communications, Inc.",286725


AT&T serves 19% of all census blocks (remember that many of them are water)

In [70]:
total_census_blocks = 11155486
df[df.hoco_final=='AT&T Inc.'].ct.values[0]/total_census_blocks

0.18710381600586473

There are 1644 different holding companies in the continental US

In [32]:
len(df)

1644

32 only serve 1 census block

In [76]:
len(df[df.ct ==1])

34

median holding company serves 669 census blocks

In [34]:
np.median(df.ct)

669.0

##  Broadband type (tech_code)

In [78]:
query = """
SELECT tech_code, COUNT(*) as ct FROM(
  SELECT
      DISTINCT(CONCAT(CAST(tech_code AS STRING), block_code)) as hoco_block,
      tech_code
  FROM broadband.fcc
  WHERE file_date = 201412)
GROUP BY tech_code
ORDER BY ct DESC
"""

df_1 = query_bq(query)

In [79]:
tech_code_dict = {10:'Asymmetric xDSL',
                    11:'ADSL2, ADSL2+',
                    12:'VDSL',
                    20:'Symmetric xDSL*',
                    30:'Other Copper Wireline' ,
                    40:'Cable Modem other than DOCSIS 1, 1.1, 2.0, 3.0, or 3.1',
                    41:'Cable Modem – DOCSIS 1, 1.1 or 2.0',
                    42:'Cable Modem – DOCSIS 3.0',
                    43:'Cable Modem – DOCSIS 3.1',
                    50:'Optical Carrier / Fiber to the end user',
                    60:'Satellite',
                    70:'Terrestrial Fixed Wireless',
                    90:'Electric Power Line',
                    0:'All Other'}

In [80]:
tech_desc = []
for i in df_1.tech_code:
    tech_desc.append(tech_code_dict[i])
df_1['tech_desc'] = tech_desc

Cable is the most popular broadband offering, followed by cable, Terrestrial Fixed Wireless DSL & Optical Fiber

In [81]:
df_1

,tech_code,ct,tech_desc
0,42,4810459,Cable Modem – DOCSIS 3.0
1,70,4086394,Terrestrial Fixed Wireless
2,10,2402590,Asymmetric xDSL
3,12,2176085,VDSL
4,11,1802126,"ADSL2, ADSL2+"
5,50,1194400,Optical Carrier / Fiber to the end user
6,41,417475,"Cable Modem – DOCSIS 1, 1.1 or 2.0"
7,30,312784,Other Copper Wireline
8,20,92101,Symmetric xDSL*
9,40,68420,"Cable Modem other than DOCSIS 1, 1.1, 2.0, 3.0..."


In [94]:
df_1_polished = df_1[['tech_desc', 'ct']]
df_1_polished.columns = ['Technology Type', 'Census Blocks']

In [97]:
df_1_polished.loc[11] = ['DSL', 2402590+2176085+1802126]
df_1_polished = df_1_polished.iloc[[11,0,1,5,6]]

df_1_polished['Technology Type'] = ['DSL',
                                   'Cable DOCSIS 3.0 - 3.1',
                                   'Terrestrial Fixed Wireless',
                                    'Fiber (FTTH)',
                                  'Cable DOCSIS 1.0 - 2.0']

In [98]:
df_1_polished

,Technology Type,Census Blocks
11,DSL,6380801
0,Cable DOCSIS 3.0 - 3.1,4810459
1,Terrestrial Fixed Wireless,4086394
5,Fiber (FTTH),1194400
6,Cable DOCSIS 1.0 - 2.0,417475


From Prof. Schulzrine's Analysis speed order is: Fiber, Cable, Terrestrial, DSL

|Technology Type | Mean | 25th Percentile|50th Percentile |75th Percentile|
| -------- | -------- | -------- | -------- |
|DSL |14.8 |6.0| 12.0 |24.0|
|Terrestrial Fixed wireless| 28.3| 8.0| 15.0 |30.0|
|Cable DOCSIS 1.0 - 2.0 |45.3 |30.0 |55.0 |90.0|
|Cable DOCSIS 3.0 - 3.1 |535.4| 200.0 |400.0| 500.0|
|Fiber (FTTH)| 658.8 |500.0 |850.0 |1000.0|


In [72]:
query = """
SELECT 
  PERCENTILE_CONT(down, 0.5) OVER() as median_down,
  PERCENTILE_CONT(up, 0.5) OVER() as median_up
FROM(
  SELECT
    block_code,
    MAX(max_ad_down) as down,
    MAX(max_ad_up) as up
  FROM broadband.fcc
  WHERE file_date = 201412
  GROUP BY block_code)
"""
df_2 = query_bq(query)

Median Maximum Offered Download Speed in all census codes with a broadband offering

In [74]:
df_2.median_down.values[0]

50.0

Median Maximum Offered Upload Speed in all census codes with a broadband offering

In [75]:
df_2.median_up.values[0]

5.0